# Optimizing Matrix-Matrix Multiplication with Blosc2

This notebook explores how different chunk sizes in **Blosc2** affect the performance of matrix-matrix multiplication. We compare automatic chunking against fixed-size chunks and analyze their impact on floating point operations per second (FLOPS) and computation time.


## Importing Required Libraries

We start by importing the necessary libraries:

- **NumPy** for matrix operations.
- **Blosc2** for handling compressed arrays and performing matrix multiplication.
- **Time** to measure performance.
- **Plotly Express** for data visualization.
- **Pandas** for data manipulation and plotting preparation.


In [ ]:
import numpy as np
import blosc2
import time
import plotly.express as px
import pandas as pd

## Defining Matrix Sizes and Compression Parameters

We define the matrix sizes to test and the chunk configurations:

- `shapes`: List of matrix dimensions to test (e.g., 1000x1000, 2000x2000, 5000x5000).
- `chunkshapes`: Two modes:
  - `None` for automatic chunking.
  - `(x, x)` for a square-fixed-size chunks.
- `cparams`: Blosc2 compression parameters using the LZ4 codec at compression level 1.

We also prepare lists to store the matrix sizes and FLOPS results for both chunking strategies.


In [ ]:
shapes = [1_000, 2_000, 5_000, 10_000]
chunkshapes = [None, (500, 500), (750, 750), (1_000, 1_000)]
cparams = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)

gflops_total = []
sizes = []
chunk_labels = []

## Generating Matrices and Performing Multiplication

For each matrix size in `shapes`:

1. Two matrices (`A` and `B`) are generated using **NumPy** with values ranging from 0 to 10.
2. These matrices are converted to **Blosc2** arrays with specified chunking.
3. Matrix multiplication is performed using `blosc2.matmul`.
4. Performance is measured in terms of floating point operations per second (GFLOPS/s).

We compare automatic chunking with fixed chunk sizes to evaluate performance differences.


In [ ]:
%%time
for N in shapes:
    shape_a = (N, N)
    shape_b = (N, N)
    size_mb = (N * N * 8) / (2 ** 20)
    total_flops = 2 * (N ** 3)

    # Generate matrices
    matrix_a_np = np.linspace(0, 10, np.prod(shape_a)).reshape(shape_a)
    matrix_b_np = np.linspace(0, 10, np.prod(shape_b)).reshape(shape_b)

    # Numpy multiplication
    t0 = time.perf_counter()
    result_numpy = np.matmul(matrix_a_np, matrix_b_np)
    numpy_time = time.perf_counter() - t0

    gflops = (total_flops / 10**9) / numpy_time

    gflops_total.append(gflops)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"Numpy: N={N}, Performance = {gflops:.2f} GFLOPS/s")

    for chunk in chunkshape:
        # Convert NumPy to Blosc2
        matrix_a_blosc2 = blosc2.asarray(matrix_a_np, cparams=cparams, chunks=chunk)
        matrix_b_blosc2 = blosc2.asarray(matrix_b_np, cparams=cparams, chunks=chunk)

        # Blosc2 multiplication
        t0 = time.perf_counter()
        result_blosc2 = blosc2.matmul(matrix_a_blosc2, matrix_b_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        # Compute GFLOPS
        gflops = (total_flops / 10**9) / blosc2_time

        sizes.append(size_mb)
        gflops_total.append(gflops)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        if chunk is None:
            print(f"Matrix A: {matrix_a_blosc2.chunks}")
            print(f"Matrix B: {matrix_b_blosc2.chunks}")
            print(f"Matrix C: {result_blosc2.chunks}")

        print(f"N={N}, Chunks = {chunk}, Performance = {gflops:.2f} GFLOPS/s")

## Visualizing GFLOPSs Performance

We use **Plotly Express** to visualize the bandwidth of the matrix-matrix multiplication for different matrix sizes and chunking strategies.

The plot shows:
- **X-axis**: Matrix size in MB.
- **Y-axis**: Floating point operations per second in GFLOPS/s.
- **Color**: Chunking strategy (Auto vs. Fixed chunks).

This helps us understand how chunking affects the efficiency of Blosc2 matrix operations.


In [ ]:
df = pd.DataFrame({
    "Matrix Size (MB)": sizes,
    "GFLOPS/s": gflops_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MB)",
              y="GFLOPS/s",
              color="Chunk Shape",
              title="Performance of Matrix-Matrix Multiplication (Blosc2 vs NumPy) in GFLOPS/s",
              labels={"value": "GFLOPS/s", "variable": "Metric"})

fig.show()

**Key observations:**
- Automatic chunking can optimize performance for smaller matrix sizes.
- Choosing square chunks of 1000x1000 can achieve the best performance for matrices of sizes greater than 2000x2000.

**Next experiment:**
We will increment the chunks' size, as we have seen that better performance can be achieved with bigger chunks.

In [ ]:
shapes = [2_000, 5_000, 10_000]
chunkshape = [None, (1_000, 1_000), (1_500, 1_500), (2_000, 2_000)]
cparams = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)

gflops_total = []
sizes = []
chunk_labels = []

In [ ]:
%%time
for N in shapes:
    shape_a = (N, N)
    shape_b = (N, N)
    size_mb = (N * N * 8) / (2 ** 20)
    total_flops = 2 * (N ** 3)

    # Generate matrices
    matrix_a_np = np.linspace(0, 10, np.prod(shape_a)).reshape(shape_a)
    matrix_b_np = np.linspace(0, 10, np.prod(shape_b)).reshape(shape_b)

    # Numpy multiplication
    t0 = time.perf_counter()
    result_numpy = np.matmul(matrix_a_np, matrix_b_np)
    numpy_time = time.perf_counter() - t0

    gflops = (total_flops / 10**9) / numpy_time

    gflops_total.append(gflops)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"Numpy: N={N}, Performance = {gflops:.2f} GFLOPS/s")

    for chunk in chunkshape:
        # Convert NumPy to Blosc2
        matrix_a_blosc2 = blosc2.asarray(matrix_a_np, cparams=cparams, chunks=chunk)
        matrix_b_blosc2 = blosc2.asarray(matrix_b_np, cparams=cparams, chunks=chunk)

        # Blosc2 multiplication
        t0 = time.perf_counter()
        result_blosc2 = blosc2.matmul(matrix_a_blosc2, matrix_b_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        # Compute GFLOPS
        gflops = (total_flops / 10**9) / blosc2_time

        sizes.append(size_mb)
        gflops_total.append(gflops)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        if chunk is None:
            print(f"Matrix A: {matrix_a_blosc2.chunks}")
            print(f"Matrix B: {matrix_b_blosc2.chunks}")
            print(f"Matrix C: {result_blosc2.chunks}")

        print(f"N={N}, Chunks = {chunk}, Performance = {gflops:.2f} GFLOPS/s")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MB)": sizes,
    "GFLOPS/s": gflops_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MB)",
              y="GFLOPS/s",
              color="Chunk Shape",
              title="Performance of Matrix-Matrix Multiplication (Blosc2 vs NumPy) in GFLOPS/s",
              labels={"value": "GFLOPS/s", "variable": "Metric"})

fig.show()

**Key observations:**
- The best performance is achieved for the biggest chunk size.
- The larger the chunk size, the higher the bandwidth.
- If the chunk size is chosen automatically, the performance is better than choosing any other chunk size. This is weird, because if chosen automatically, chunks of size 1000x1000 are chosen, which is the same size as the fixed chunks.

**Next experiment:**
We will increment the chunks' size again, as we have seen that better performance can be achieved with bigger chunks.

Precision simple

In [ ]:
shapes = [1_000, 2_000, 5_000, 7_000, 8_000, 9_000, 10_000, 12_000, 14_000, 16_000, 18_000]
chunkshape = [None, (2_000, 2_000), (5_000, 5_000), (10_000, 10_000), (12_000, 12_000)]
cparams = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)

gflops_total = []
sizes = []
chunk_labels = []

In [ ]:
%%time
for N in shapes:
    shape_a = (N, N)
    shape_b = (N, N)
    size_mb = (N * N * 8) / (2 ** 20)
    total_flops = 2 * (N ** 3)

    # Generate matrices
    matrix_a_np = np.linspace(0, 1, np.prod(shape_a), dtype=np.float32).reshape(shape_a)
    matrix_b_np = np.linspace(0, 1, np.prod(shape_b), dtype=np.float32).reshape(shape_b)

    # Numpy multiplication
    t0 = time.perf_counter()
    result_numpy = np.matmul(matrix_a_np, matrix_b_np)
    numpy_time = time.perf_counter() - t0

    gflops = (total_flops / 10**9) / numpy_time

    gflops_total.append(gflops)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"Numpy: N={N}, Performance = {gflops:.2f} GFLOPS/s")

    for chunk in chunkshape:
        # Convert NumPy to Blosc2
        matrix_a_blosc2 = blosc2.asarray(matrix_a_np, cparams=cparams, chunks=chunk)
        matrix_b_blosc2 = blosc2.asarray(matrix_b_np, cparams=cparams, chunks=chunk)

        # Blosc2 multiplication
        t0 = time.perf_counter()
        result_blosc2 = blosc2.matmul(matrix_a_blosc2, matrix_b_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        # Compute GFLOPS
        gflops = (total_flops / 10**9) / blosc2_time

        sizes.append(size_mb)
        gflops_total.append(gflops)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        if chunk is None:
            print(f"Matrix A: {matrix_a_blosc2.chunks}")
            print(f"Matrix B: {matrix_b_blosc2.chunks}")
            print(f"Matrix C: {result_blosc2.chunks}")

        print(f"N={N}, Chunks = {chunk}, Performance = {gflops:.2f} GFLOPS/s, CRatio = {result_blosc2.schunk.cratio:.2f}x")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MBs)": sizes,
    "GFLOPS/s": gflops_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MBs)",
              y="GFLOPS/s",
              color="Chunk Shape",
              title="Float32 Matrix Multiplication (Blosc2 vs NumPy)",
              labels={"value": "GFLOPS/s", "variable": "Metric"})

fig.for_each_trace(lambda t: t.update(line=dict(color='darkgray', dash='dash')) if t.name == 'NumPy' else ())

fig.show()

In [ ]:
shapes = [1_000, 2_000, 5_000, 7_000, 8_000, 9_000, 10_000, 12_000, 14_000, 16_000, 18_000]
chunkshape = [None, (2_000, 2_000), (5_000, 5_000), (10_000, 10_000), (12_000, 12_000)]
cparams = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)

gflops_total = []
sizes = []
chunk_labels = []

In [ ]:
%%time
for N in shapes:
    shape_a = (N, N)
    shape_b = (N, N)
    size_mb = (N * N * 8) / (2 ** 20)
    total_flops = 2 * (N ** 3)

    # Generate matrices
    matrix_a_np = np.linspace(0, 1, np.prod(shape_a), dtype=np.float32).reshape(shape_a)
    matrix_b_np = np.linspace(0, 1, np.prod(shape_b), dtype=np.float32).reshape(shape_b)

    # Numpy multiplication
    t0 = time.perf_counter()
    result_numpy = np.matmul(matrix_a_np, matrix_b_np)
    numpy_time = time.perf_counter() - t0

    gflops = (total_flops / 10**9) / numpy_time

    gflops_total.append(gflops)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"Numpy: N={N}, Performance = {gflops:.2f} GFLOPS/s")

    for chunk in chunkshape:
        # Convert NumPy to Blosc2
        matrix_a_blosc2 = blosc2.asarray(matrix_a_np, cparams=cparams, chunks=chunk)
        matrix_b_blosc2 = blosc2.asarray(matrix_b_np, cparams=cparams, chunks=chunk)

        # Blosc2 multiplication
        t0 = time.perf_counter()
        result_blosc2 = blosc2.matmul(matrix_a_blosc2, matrix_b_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        # Compute GFLOPS
        gflops = (total_flops / 10**9) / blosc2_time

        sizes.append(size_mb)
        gflops_total.append(gflops)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        if chunk is None:
            print(f"Matrix A: {matrix_a_blosc2.chunks}")
            print(f"Matrix B: {matrix_b_blosc2.chunks}")
            print(f"Matrix C: {result_blosc2.chunks}")

        print(f"N={N}, Chunks = {chunk}, Performance = {gflops:.2f} GFLOPS/s, CRatio = {result_blosc2.schunk.cratio:.2f}x")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MBs)": sizes,
    "GFLOPS/s": gflops_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MBs)",
              y="GFLOPS/s",
              color="Chunk Shape",
              title="Float64 Matrix Multiplication (Blosc2 vs NumPy)",
              labels={"value": "GFLOPS/s", "variable": "Metric"})

fig.for_each_trace(lambda t: t.update(line=dict(color='darkgray', dash='dash')) if t.name == 'NumPy' else ())

fig.show()

Per algun motiu el quadrat es millor que el rectangular.

**Key observations:**
- The best performance is achieved for the biggest chunk size and matrices of sizes greater than 5000x5000.
- The larger the chunk size, the higher the bandwidth.
- We can see that the performance on the matrix if size 12000x12000, chunks of size 2000 is better than 2500. This could be because 12000 is not divisible by 2500 but it is divisible by 2000.

**Next experiment:**
We are going to try with the same sizes for matrices and a square chunk size of 6000 to see if it improves the performance for that last matrix size.
We will also remove chunk sizes of 1000 and 2000, and add a chunk size which will be the same size as the matrix.

In [ ]:
shapes = [5_000, 6_000, 10_000, 12_000]
chunkshape = [None, (1_000, 1_000), (2_000, 2_000), (2_500, 2_500), (3_000, 3_000), (5_000, 5_000)]
cparams = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)

gflops_total = []
sizes = []
chunk_labels = []

In [ ]:
%%time
for N in shapes:
    shape_a = (N, N)
    shape_b = (N, N)
    size_mb = (N * N * 8) / (2 ** 20)
    total_flops = 2 * (N ** 3)

    # Generate matrices
    matrix_a_np = np.linspace(0, 10, np.prod(shape_a)).reshape(shape_a)
    matrix_b_np = np.linspace(0, 10, np.prod(shape_b)).reshape(shape_b)

    # Numpy multiplication
    t0 = time.perf_counter()
    result_numpy = np.matmul(matrix_a_np, matrix_b_np)
    numpy_time = time.perf_counter() - t0

    gflops = (total_flops / 10**9) / numpy_time

    gflops_total.append(gflops)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"Numpy: N={N}, Performance = {gflops:.2f} GFLOPS/s")

    for chunk in chunkshape:
        # Convert NumPy to Blosc2
        matrix_a_blosc2 = blosc2.asarray(matrix_a_np, cparams=cparams, chunks=chunk)
        matrix_b_blosc2 = blosc2.asarray(matrix_b_np, cparams=cparams, chunks=chunk)

        # Blosc2 multiplication
        t0 = time.perf_counter()
        result_blosc2 = blosc2.matmul(matrix_a_blosc2, matrix_b_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        # Compute GFLOPS
        gflops = (total_flops / 10**9) / blosc2_time

        sizes.append(size_mb)
        gflops_total.append(gflops)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        if chunk is None:
            print(f"Matrix A: {matrix_a_blosc2.chunks}")
            print(f"Matrix B: {matrix_b_blosc2.chunks}")
            print(f"Matrix C: {result_blosc2.chunks}")

        print(f"N={N}, Chunks = {chunk}, Performance = {gflops:.2f} GFLOPS/s")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MB)": sizes,
    "GFLOPS/s": gflops_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MB)",
              y="GFLOPS/s",
              color="Chunk Shape",
              title="Performance of Matrix-Matrix Multiplication (Blosc2 vs NumPy) in GFLOPS/s",
              labels={"value": "GFLOPS/s", "variable": "Metric"})

fig.show()

## Second type of benchmarks

We are going to experiment with other type of chunks, we will see if automatic performance is better than the same chunk size but inverted.

In [ ]:
shapes = [5_000, 6_000, 10_000, 12_000]
chunkshape = [None, (1_000, 1_000), (2_000, 2_000), (2_500, 2_500), (3_000, 3_000), (5_000, 5_000)]
cparams = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)

gflops_total = []
sizes = []
chunk_labels = []

In [ ]:
%%time
for N in shapes:
    shape_a = (N, N)
    shape_b = (N, N)
    size_mb = (N * N * 8) / (2 ** 20)
    total_flops = 2 * (N ** 3)

    # Generate matrices
    matrix_a_np = np.linspace(0, 10, np.prod(shape_a)).reshape(shape_a)
    matrix_b_np = np.linspace(0, 10, np.prod(shape_b)).reshape(shape_b)

    # Numpy multiplication
    t0 = time.perf_counter()
    result_numpy = np.matmul(matrix_a_np, matrix_b_np)
    numpy_time = time.perf_counter() - t0

    gflops = (total_flops / 10 ** 9) / numpy_time

    gflops_total.append(gflops)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"Numpy: N={N}, Performance = {gflops:.2f} GFLOPS/s")

    for chunk in chunkshape:
        # Convert NumPy to Blosc2
        matrix_a_blosc2 = blosc2.asarray(matrix_a_np, cparams=cparams)
        matrix_b_blosc2 = blosc2.asarray(matrix_b_np, cparams=cparams)

        # Blosc2 multiplication
        t0 = time.perf_counter()
        result_blosc2 = blosc2.matmul(matrix_a_blosc2, matrix_b_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        # Compute GFLOPS
        gflops = (total_flops / 10**9) / blosc2_time

        sizes.append(size_mb)
        gflops_total.append(gflops)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        if chunk is None:
            print(f"Matrix A: {matrix_a_blosc2.chunks}")
            print(f"Matrix B: {matrix_b_blosc2.chunks}")
            print(f"Matrix C: {result_blosc2.chunks}")

        print(f"N={N}, Chunks = {chunk}, Performance = {gflops:.2f} GFLOPS/s")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MB)": sizes,
    "GFLOPS/s": gflops_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MB)",
              y="GFLOPS/s",
              color="Chunk Shape",
              title="Performance of Matrix-Matrix Multiplication (Blosc2 vs NumPy) in GFLOPS/s",
              labels={"value": "GFLOPS/s", "variable": "Metric"})

fig.show()

Let's try creating the second matrix to be multiplied, b, with the same chunks of matrix a but inverted.

In [ ]:
shapes = [5_000, 6_000, 10_000]
chunkshape = [None, (1_000, 1_000), (2_000, 2_000), (3_000, 3_000), (5_000, 5_000)]
cparams = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)

gflops_total = []
sizes = []
chunk_labels = []

In [ ]:
%%time
for N in shapes:
    shape_a = (N, N)
    shape_b = (N, N)
    size_mb = (N * N * 8) / (2 ** 20)
    total_flops = 2 * (N ** 3)

    # Generate matrices
    matrix_a_np = np.linspace(0, 10, np.prod(shape_a)).reshape(shape_a)
    matrix_b_np = np.linspace(0, 10, np.prod(shape_b)).reshape(shape_b)

    # Numpy multiplication
    t0 = time.perf_counter()
    result_numpy = np.matmul(matrix_a_np, matrix_b_np)
    numpy_time = time.perf_counter() - t0

    gflops = (total_flops / 10**9) / numpy_time

    gflops_total.append(gflops)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"Numpy: N={N}, Performance = {gflops:.2f} GFLOPS/s")

    for chunk in chunkshape:
        # Convert NumPy to Blosc2
        matrix_a_blosc2 = blosc2.asarray(matrix_a_np, cparams=cparams)
        matrix_b_blosc2 = blosc2.asarray(matrix_b_np, cparams=cparams, chunks=chunk)

        # Blosc2 multiplication
        t0 = time.perf_counter()
        result_blosc2 = blosc2.matmul(matrix_a_blosc2, matrix_b_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        # Compute GFLOPS
        gflops = (total_flops / 10**9) / blosc2_time

        sizes.append(size_mb)
        gflops_total.append(gflops)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        if chunk is None:
            print(f"Matrix A: {matrix_a_blosc2.chunks}")
            print(f"Matrix B: {matrix_b_blosc2.chunks}")
            print(f"Matrix C: {result_blosc2.chunks}")

        print(f"N={N}, Chunks = {chunk}, Performance = {gflops:.2f} GFLOPS/s")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MB)": sizes,
    "GFLOPS/s": gflops_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MB)",
              y="GFLOPS/s",
              color="Chunk Shape",
              title="Performance of Matrix-Matrix Multiplication (Blosc2 vs NumPy) in GFLOPS/s",
              labels={"value": "GFLOPS/s", "variable": "Metric"})

fig.show()